In [1]:
import pandas as pd
import numpy as np
from shapely.geometry import Point
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [2]:
train_df = pd.read_csv("data/CTA_L_Ridership_Monthly_by_Blockgroup.csv")

In [3]:
bus_df = gpd.read_file("data/area_CTA_routes.geojson")

In [4]:
lic_df = pd.read_csv("data/business_licenses_by_blockgroup.csv")

In [6]:
print(train_df.dtypes)
train_df.head()

blockgroup          int64
month_beginning    object
train_rides         int64
dtype: object


,blockgroup,month_beginning,train_rides
0,170310101001,01/01/2001,377399
1,170310101001,01/01/2002,356035
2,170310101001,01/01/2003,340125
3,170310101001,01/01/2004,324526
4,170310101001,01/01/2005,325312


In [7]:
print(bus_df.dtypes)
bus_df.head()

blockgroup              object
year                     int64
month                    int64
prior_year               int64
pri_neigh               object
Population               int64
pop_change             float64
Median Income          float64
income_change          float64
Median Age             float64
age_change             float64
WorkTransitCount         int64
wt_count_change        float64
WorkTransitPercent     float64
wt_perc_change         float64
count_of_routes          int64
rt_count_change        float64
MonthTotal             float64
geometry              geometry
dtype: object


,blockgroup,year,month,prior_year,pri_neigh,Population,pop_change,Median Income,income_change,Median Age,age_change,WorkTransitCount,wt_count_change,WorkTransitPercent,wt_perc_change,count_of_routes,rt_count_change,MonthTotal,geometry
0,170310101002,2018,1,2017,Rogers Park,2197,0.012442,21222.0,0.024426,31.4,0.003195,1087,-0.101653,43.9,-0.132411,2,0.0,13420.077747,"POLYGON ((-87.66950 42.01936, -87.66963 42.019..."
1,170310101002,2018,2,2017,Rogers Park,2197,0.012442,21222.0,0.024426,31.4,0.003195,1087,-0.101653,43.9,-0.132411,2,0.0,12733.663576,"POLYGON ((-87.66950 42.01936, -87.66963 42.019..."
2,170310101002,2018,3,2017,Rogers Park,2197,0.012442,21222.0,0.024426,31.4,0.003195,1087,-0.101653,43.9,-0.132411,2,0.0,14869.838672,"POLYGON ((-87.66950 42.01936, -87.66963 42.019..."
3,170310101002,2018,4,2017,Rogers Park,2197,0.012442,21222.0,0.024426,31.4,0.003195,1087,-0.101653,43.9,-0.132411,2,0.0,14445.116485,"POLYGON ((-87.66950 42.01936, -87.66963 42.019..."
4,170310101002,2018,5,2017,Rogers Park,2197,0.012442,21222.0,0.024426,31.4,0.003195,1087,-0.101653,43.9,-0.132411,2,0.0,14974.533570,"POLYGON ((-87.66950 42.01936, -87.66963 42.019..."


In [8]:
print(lic_df.dtypes)
lic_df.head()

blockgroup           int64
month-year          object
active               int64
new                float64
month                int64
year                 int64
prev_month-year     object
prev_yr_active     float64
prev_yr_new        float64
%_change_active    float64
%_change_new       float64
dtype: object


,blockgroup,month-year,active,new,month,year,prev_month-year,prev_yr_active,prev_yr_new,%_change_active,%_change_new
0,170310101001,2014-01-01,4,0.0,1,2014,NaN,NaN,NaN,0.0,0.0
1,170310101002,2014-01-01,22,0.0,1,2014,NaN,NaN,NaN,0.0,0.0
2,170310101003,2014-01-01,12,0.0,1,2014,NaN,NaN,NaN,0.0,0.0
3,170310102011,2014-01-01,2,0.0,1,2014,NaN,NaN,NaN,0.0,0.0
4,170310102012,2014-01-01,35,0.0,1,2014,NaN,NaN,NaN,0.0,0.0


### Train Data

In [9]:
train_df['month_beginning'] = pd.to_datetime(train_df['month_beginning'])
print(train_df.dtypes)
train_df.head()

blockgroup                  int64
month_beginning    datetime64[ns]
train_rides                 int64
dtype: object


,blockgroup,month_beginning,train_rides
0,170310101001,2001-01-01,377399
1,170310101001,2002-01-01,356035
2,170310101001,2003-01-01,340125
3,170310101001,2004-01-01,324526
4,170310101001,2005-01-01,325312


In [10]:
train_df['month'] = train_df['month_beginning'].dt.month
train_df['year'] = train_df['month_beginning'].dt.year
print(train_df.dtypes)
train_df

blockgroup                  int64
month_beginning    datetime64[ns]
train_rides                 int64
month                       int64
year                        int64
dtype: object


,blockgroup,month_beginning,train_rides,month,year
0,170310101001,2001-01-01,377399,1,2001
1,170310101001,2002-01-01,356035,1,2002
2,170310101001,2003-01-01,340125,1,2003
3,170310101001,2004-01-01,324526,1,2004
4,170310101001,2005-01-01,325312,1,2005
...,...,...,...,...,...
309279,170318439002,2015-12-01,33052,12,2015
309280,170318439002,2016-12-01,27594,12,2016
309281,170318439002,2017-12-01,26317,12,2017
309282,170318439002,2018-12-01,21889,12,2018


In [11]:
train_df.year.value_counts()

2018    16080
2017    16080
2016    16080
2015    16080
2019    16080
2010    16068
2009    16068
2005    16068
2006    16068
2007    16068
2008    16068
2012    16068
2011    16068
2003    16068
2002    16068
2014    16068
2004    16068
2001    16063
2013    15985
2020     4020
Name: year, dtype: int64

In [23]:
train_df_small = train_df[(train_df.year >= 2014) & (train_df.year < 2019)].copy()
train_df_small

,blockgroup,month_beginning,train_rides,month,year
13,170310101001,2014-01-01,355833,1,2014
14,170310101001,2015-01-01,383403,1,2015
15,170310101001,2016-01-01,368646,1,2016
16,170310101001,2017-01-01,360784,1,2017
17,170310101001,2018-01-01,342557,1,2018
...,...,...,...,...,...
309278,170318439002,2014-12-01,31911,12,2014
309279,170318439002,2015-12-01,33052,12,2015
309280,170318439002,2016-12-01,27594,12,2016
309281,170318439002,2017-12-01,26317,12,2017


In [24]:
train_df_small.year.value_counts()

2018    16080
2017    16080
2016    16080
2015    16080
2014    16068
Name: year, dtype: int64

### Merge Train and Licenses Data

In [25]:
print(train_df_small.shape)
print(lic_df.shape)

(80388, 5)
(129008, 11)


In [26]:
print(train_df_small.blockgroup.nunique())
print(lic_df.blockgroup.nunique())

1340
2175


In [19]:
lic_df['month-year'] = pd.to_datetime(lic_df['month-year'])
lic_df['prev_month-year'] = pd.to_datetime(lic_df['prev_month-year'])
print(lic_df.dtypes)
lic_df.head()

blockgroup                  int64
month-year         datetime64[ns]
active                      int64
new                       float64
month                       int64
year                        int64
prev_month-year    datetime64[ns]
prev_yr_active            float64
prev_yr_new               float64
%_change_active           float64
%_change_new              float64
dtype: object


,blockgroup,month-year,active,new,month,year,prev_month-year,prev_yr_active,prev_yr_new,%_change_active,%_change_new
0,170310101001,2014-01-01,4,0.0,1,2014,NaT,NaN,NaN,0.0,0.0
1,170310101002,2014-01-01,22,0.0,1,2014,NaT,NaN,NaN,0.0,0.0
2,170310101003,2014-01-01,12,0.0,1,2014,NaT,NaN,NaN,0.0,0.0
3,170310102011,2014-01-01,2,0.0,1,2014,NaT,NaN,NaN,0.0,0.0
4,170310102012,2014-01-01,35,0.0,1,2014,NaT,NaN,NaN,0.0,0.0


In [28]:
train_df_small.rename(columns={'month_beginning': 'month-year'}, inplace=True)
train_df_small.head()

,blockgroup,month-year,train_rides,month,year
13,170310101001,2014-01-01,355833,1,2014
14,170310101001,2015-01-01,383403,1,2015
15,170310101001,2016-01-01,368646,1,2016
16,170310101001,2017-01-01,360784,1,2017
17,170310101001,2018-01-01,342557,1,2018


In [33]:
merged_df = lic_df.merge(train_df_small[['blockgroup', 'month-year', 'train_rides']], 
                         how='outer', on=['blockgroup', 'month-year'])
merged_df

,blockgroup,month-year,active,new,month,year,prev_month-year,prev_yr_active,prev_yr_new,%_change_active,%_change_new,train_rides
0,170310101001,2014-01-01,4.0,0.0,1.0,2014.0,NaT,NaN,NaN,0.0,0.0,355833.0
1,170310101002,2014-01-01,22.0,0.0,1.0,2014.0,NaT,NaN,NaN,0.0,0.0,355833.0
2,170310101003,2014-01-01,12.0,0.0,1.0,2014.0,NaT,NaN,NaN,0.0,0.0,324378.0
3,170310102011,2014-01-01,2.0,0.0,1.0,2014.0,NaT,NaN,NaN,0.0,0.0,324378.0
4,170310102012,2014-01-01,35.0,0.0,1.0,2014.0,NaT,NaN,NaN,0.0,0.0,324378.0
...,...,...,...,...,...,...,...,...,...,...,...,...
130131,170318392003,2017-11-01,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,318719.0
130132,170318392003,2018-11-01,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,305642.0
130133,170318392003,2014-12-01,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,289386.0
130134,170318392003,2017-12-01,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,261953.0


In [41]:
# Checks
print('These should be match:')
print((merged_df.train_rides.isnull().sum()) - (lic_df.shape[0] - train_df_small.shape[0]))
print(merged_df.shape[0] - lic_df.shape[0])

print('Other checks:')
print(merged_df.blockgroup.nunique())
print('This should be 0:', lic_df[lic_df['blockgroup'] == '170318392003'].shape[0])

These should be match:
1128
1128
Other checks:
2180
This should be 0: 0


In [42]:
merged_df.dtypes

blockgroup                  int64
month-year         datetime64[ns]
active                    float64
new                       float64
month                     float64
year                      float64
prev_month-year    datetime64[ns]
prev_yr_active            float64
prev_yr_new               float64
%_change_active           float64
%_change_new              float64
train_rides               float64
dtype: object

In [48]:
# Fix columns
merged_df['active'].fillna(0, inplace=True)
merged_df['new'].fillna(0, inplace=True)
merged_df['%_change_active'].fillna(0, inplace=True)
merged_df['%_change_new'].fillna(0, inplace=True)

merged_df['month'] = merged_df['month-year'].dt.month
merged_df['year'] = merged_df['month-year'].dt.year
merged_df

,blockgroup,month-year,active,new,month,year,prev_month-year,prev_yr_active,prev_yr_new,%_change_active,%_change_new,train_rides
0,170310101001,2014-01-01,4.0,0.0,1,2014,NaT,NaN,NaN,0.0,0.0,355833.0
1,170310101002,2014-01-01,22.0,0.0,1,2014,NaT,NaN,NaN,0.0,0.0,355833.0
2,170310101003,2014-01-01,12.0,0.0,1,2014,NaT,NaN,NaN,0.0,0.0,324378.0
3,170310102011,2014-01-01,2.0,0.0,1,2014,NaT,NaN,NaN,0.0,0.0,324378.0
4,170310102012,2014-01-01,35.0,0.0,1,2014,NaT,NaN,NaN,0.0,0.0,324378.0
...,...,...,...,...,...,...,...,...,...,...,...,...
130131,170318392003,2017-11-01,0.0,0.0,11,2017,NaT,NaN,NaN,0.0,0.0,318719.0
130132,170318392003,2018-11-01,0.0,0.0,11,2018,NaT,NaN,NaN,0.0,0.0,305642.0
130133,170318392003,2014-12-01,0.0,0.0,12,2014,NaT,NaN,NaN,0.0,0.0,289386.0
130134,170318392003,2017-12-01,0.0,0.0,12,2017,NaT,NaN,NaN,0.0,0.0,261953.0


In [58]:
merged_df['blockgroup'] = merged_df['blockgroup'].astype(str)
merged_df.dtypes

blockgroup                 object
month-year         datetime64[ns]
active                    float64
new                       float64
month                       int64
year                        int64
prev_month-year    datetime64[ns]
prev_yr_active            float64
prev_yr_new               float64
%_change_active           float64
%_change_new              float64
train_rides               float64
dtype: object

### Merge Bus Routes Info

In [49]:
bus_df.shape

(92644, 19)

In [50]:
print(bus_df.blockgroup.nunique())
print(bus_df.year.value_counts())

1937
2018    23232
2017    23164
2016    23124
2015    23124
Name: year, dtype: int64


In [55]:
# Add month-year column to merge on
bus_df['month-year'] = pd.to_datetime(bus_df[['month', 'year']].assign(Day=1))
bus_df

,blockgroup,year,month,prior_year,pri_neigh,Population,pop_change,Median Income,income_change,Median Age,age_change,WorkTransitCount,wt_count_change,WorkTransitPercent,wt_perc_change,count_of_routes,rt_count_change,MonthTotal,geometry,month-year
0,170310101002,2018,1,2017,Rogers Park,2197,0.012442,21222.0,0.024426,31.4,0.003195,1087,-0.101653,43.9,-0.132411,2,0.0,13420.077747,"POLYGON ((-87.66950 42.01936, -87.66963 42.019...",2018-01-01
1,170310101002,2018,2,2017,Rogers Park,2197,0.012442,21222.0,0.024426,31.4,0.003195,1087,-0.101653,43.9,-0.132411,2,0.0,12733.663576,"POLYGON ((-87.66950 42.01936, -87.66963 42.019...",2018-02-01
2,170310101002,2018,3,2017,Rogers Park,2197,0.012442,21222.0,0.024426,31.4,0.003195,1087,-0.101653,43.9,-0.132411,2,0.0,14869.838672,"POLYGON ((-87.66950 42.01936, -87.66963 42.019...",2018-03-01
3,170310101002,2018,4,2017,Rogers Park,2197,0.012442,21222.0,0.024426,31.4,0.003195,1087,-0.101653,43.9,-0.132411,2,0.0,14445.116485,"POLYGON ((-87.66950 42.01936, -87.66963 42.019...",2018-04-01
4,170310101002,2018,5,2017,Rogers Park,2197,0.012442,21222.0,0.024426,31.4,0.003195,1087,-0.101653,43.9,-0.132411,2,0.0,14974.533570,"POLYGON ((-87.66950 42.01936, -87.66963 42.019...",2018-05-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92639,170319801001,2015,8,2014,Garfield Ridge,0,NaN,NaN,NaN,NaN,NaN,0,NaN,0.0,NaN,10,0.0,0.000000,"POLYGON ((-87.75442 41.77819, -87.75516 41.778...",2015-08-01
92640,170319801001,2015,9,2014,Garfield Ridge,0,NaN,NaN,NaN,NaN,NaN,0,NaN,0.0,NaN,10,0.0,0.000000,"POLYGON ((-87.75442 41.77819, -87.75516 41.778...",2015-09-01
92641,170319801001,2015,10,2014,Garfield Ridge,0,NaN,NaN,NaN,NaN,NaN,0,NaN,0.0,NaN,10,0.0,0.000000,"POLYGON ((-87.75442 41.77819, -87.75516 41.778...",2015-10-01
92642,170319801001,2015,11,2014,Garfield Ridge,0,NaN,NaN,NaN,NaN,NaN,0,NaN,0.0,NaN,10,0.0,0.000000,"POLYGON ((-87.75442 41.77819, -87.75516 41.778...",2015-11-01


In [59]:
merged_df2 = bus_df.merge(merged_df, how='outer', on=['blockgroup', 'month-year'])
print(type(merged_df2))
merged_df2

<class 'geopandas.geodataframe.GeoDataFrame'>


,blockgroup,year_x,month_x,prior_year,pri_neigh,Population,pop_change,Median Income,income_change,Median Age,...,active,new,month_y,year_y,prev_month-year,prev_yr_active,prev_yr_new,%_change_active,%_change_new,train_rides
0,170310101002,2018.0,1.0,2017.0,Rogers Park,2197.0,0.012442,21222.0,0.024426,31.4,...,18.0,1.0,1.0,2018.0,2017-01-01,23.0,1.0,0.782609,1.0,342557.0
1,170310101002,2018.0,2.0,2017.0,Rogers Park,2197.0,0.012442,21222.0,0.024426,31.4,...,18.0,0.0,2.0,2018.0,2017-02-01,23.0,0.0,0.782609,0.0,326069.0
2,170310101002,2018.0,3.0,2017.0,Rogers Park,2197.0,0.012442,21222.0,0.024426,31.4,...,18.0,0.0,3.0,2018.0,2017-03-01,21.0,0.0,0.857143,0.0,365391.0
3,170310101002,2018.0,4.0,2017.0,Rogers Park,2197.0,0.012442,21222.0,0.024426,31.4,...,17.0,0.0,4.0,2018.0,2017-04-01,20.0,0.0,0.850000,0.0,352160.0
4,170310101002,2018.0,5.0,2017.0,Rogers Park,2197.0,0.012442,21222.0,0.024426,31.4,...,17.0,0.0,5.0,2018.0,2017-05-01,20.0,0.0,0.850000,0.0,373607.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130699,170318392001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,12.0,2014.0,NaT,NaN,NaN,0.000000,0.0,165834.0
130700,170318392003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,9.0,2014.0,NaT,NaN,NaN,0.000000,0.0,372900.0
130701,170318392003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,10.0,2014.0,NaT,NaN,NaN,0.000000,0.0,377996.0
130702,170318392003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,11.0,2014.0,NaT,NaN,NaN,0.000000,0.0,308653.0


In [60]:
merged_df2.dtypes

blockgroup                    object
year_x                       float64
month_x                      float64
prior_year                   float64
pri_neigh                     object
Population                   float64
pop_change                   float64
Median Income                float64
income_change                float64
Median Age                   float64
age_change                   float64
WorkTransitCount             float64
wt_count_change              float64
WorkTransitPercent           float64
wt_perc_change               float64
count_of_routes              float64
rt_count_change              float64
MonthTotal                   float64
geometry                    geometry
month-year            datetime64[ns]
active                       float64
new                          float64
month_y                      float64
year_y                       float64
prev_month-year       datetime64[ns]
prev_yr_active               float64
prev_yr_new                  float64
%

In [64]:
# Fix duplicate columns
merged_df2.drop(columns=['month_y', 'year_y'], inplace=True)
merged_df2.rename(columns={'month_x': 'month', 'year_x': 'year'}, inplace=True)
merged_df2.head()

,blockgroup,year,month,prior_year,pri_neigh,Population,pop_change,Median Income,income_change,Median Age,...,geometry,month-year,active,new,prev_month-year,prev_yr_active,prev_yr_new,%_change_active,%_change_new,train_rides
0,170310101002,2018.0,1.0,2017.0,Rogers Park,2197.0,0.012442,21222.0,0.024426,31.4,...,"POLYGON ((-87.66950 42.01936, -87.66963 42.019...",2018-01-01,18.0,1.0,2017-01-01,23.0,1.0,0.782609,1.0,342557.0
1,170310101002,2018.0,2.0,2017.0,Rogers Park,2197.0,0.012442,21222.0,0.024426,31.4,...,"POLYGON ((-87.66950 42.01936, -87.66963 42.019...",2018-02-01,18.0,0.0,2017-02-01,23.0,0.0,0.782609,0.0,326069.0
2,170310101002,2018.0,3.0,2017.0,Rogers Park,2197.0,0.012442,21222.0,0.024426,31.4,...,"POLYGON ((-87.66950 42.01936, -87.66963 42.019...",2018-03-01,18.0,0.0,2017-03-01,21.0,0.0,0.857143,0.0,365391.0
3,170310101002,2018.0,4.0,2017.0,Rogers Park,2197.0,0.012442,21222.0,0.024426,31.4,...,"POLYGON ((-87.66950 42.01936, -87.66963 42.019...",2018-04-01,17.0,0.0,2017-04-01,20.0,0.0,0.850000,0.0,352160.0
4,170310101002,2018.0,5.0,2017.0,Rogers Park,2197.0,0.012442,21222.0,0.024426,31.4,...,"POLYGON ((-87.66950 42.01936, -87.66963 42.019...",2018-05-01,17.0,0.0,2017-05-01,20.0,0.0,0.850000,0.0,373607.0


### Final Checks

In [65]:
print(type(merged_df2))
print(merged_df2.blockgroup.nunique())
print(merged_df2['month-year'].nunique())
print(merged_df.year.value_counts())

<class 'geopandas.geodataframe.GeoDataFrame'>
2186
60
2014    26063
2016    26028
2015    26020
2018    26017
2017    26008
Name: year, dtype: int64


In [68]:
merged_df2[merged_df2.geometry.isnull()]

,blockgroup,year,month,prior_year,pri_neigh,Population,pop_change,Median Income,income_change,Median Age,...,geometry,month-year,active,new,prev_month-year,prev_yr_active,prev_yr_new,%_change_active,%_change_new,train_rides
92644,170310101001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,None,2014-01-01,4.0,0.0,NaT,NaN,NaN,0.0,0.0,355833.0
92645,170310101002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,None,2014-01-01,22.0,0.0,NaT,NaN,NaN,0.0,0.0,355833.0
92646,170310101003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,None,2014-01-01,12.0,0.0,NaT,NaN,NaN,0.0,0.0,324378.0
92647,170310102011,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,None,2014-01-01,2.0,0.0,NaT,NaN,NaN,0.0,0.0,324378.0
92648,170310102012,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,None,2014-01-01,35.0,0.0,NaT,NaN,NaN,0.0,0.0,324378.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130699,170318392001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,None,2014-12-01,0.0,0.0,NaT,NaN,NaN,0.0,0.0,165834.0
130700,170318392003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,None,2014-09-01,0.0,0.0,NaT,NaN,NaN,0.0,0.0,372900.0
130701,170318392003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,None,2014-10-01,0.0,0.0,NaT,NaN,NaN,0.0,0.0,377996.0
130702,170318392003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,None,2014-11-01,0.0,0.0,NaT,NaN,NaN,0.0,0.0,308653.0


In [67]:
merged_df2.to_file("data/MERGED_DATA.geojson", driver='GeoJSON')